In [4]:
import numpy as np
import pandas as pd
import os
import math
from time import time
#定义几个函数
# def find_key(keys,obj):  二分查找user id为obj的用户是否在训练集用户列表keys中出现
# def softmax(vec): 9类别的时长转softmax概率，时长太大，指数溢出暂时不用
# def cnt2pro(vec): 9类别的时长转概率，分别除以9类别时长总和
# def close_txt(id_cnt_files): 一次性关闭多个txt文件，id_cnt_files为文件列表
# def getfiles(mode): 返回文件名列表，mode："train", "val", "test"

def find_key(keys,obj):
    len_keys=len(keys)
    l=0
    r=len_keys-1
    while True:
        if r-l<3:
            break
        mid=(l+r)//2
        if keys[mid]==obj:
            return True
        if keys[mid]<obj:
            l=mid+1
        else:
            r=mid+1
    for k in range(l,r+1):
        if keys[k]==obj:
            return True
    return False    
def softmax(vec):
    return_ans=np.zeros([vec.shape[0]])
    for k in range(vec.shape[0]):
        SUM=0.0
        for n in range(vec.shape[0]):
            SUM+=math.exp(vec[n]-vec[k])
        return_ans=1.0/SUM
    return return_ans
def cnt2pro(vec):
    return_ans=np.zeros([vec.shape[0]])
    SUM=0.0
    for n in range(vec.shape[0]):
        SUM+=vec[n]
    for k in range(vec.shape[0]):
        return_ans[k]=vec[k]/SUM
    return return_ans
def close_txt(id_cnt_files):
    for file in id_cnt_files:
        file.close()
def getfiles(mode):
    #嵌套函数
    def get_name_from_txt(file_name):
        name_id=[]
        f=open(file_name)
        lines=f.readlines()
        for line in lines:
            s=line.split(".jpg")[0]
            s=s.split("/")[-1]
            name_id.append(s)
        f.close()
        return name_id
    ##
    return_ans=[]
    if mode=="test":
        for i in range(2):
            path="/root/userfolder/luotao/final/test_visit_"+str(i)+"/"+str(i)+"/"
            files=os.listdir(path)
            for file in files:
                return_ans.append(path+file)
        return_ans.sort()
        return return_ans
    elif mode=="train":
        file_name="/root/userfolder/luotao/final_code/data_txt/train.txt"
    elif mode=="val":
        file_name="/root/userfolder/luotao/final_code/data_txt/val.txt"
    name_id=get_name_from_txt(file_name)
    name2file={}
    for i in range(10):
        path="/root/userfolder/luotao/final/train_visit_"+str(i)+"/"+str(i)+"/"
        files=os.listdir(path)
        for file in files:
            s=file.split(".txt")[0]
            name2file[s]=path+file
    return_ans=[]
    for name in name_id:
        return_ans.append(name2file[name])
    return return_ans

In [2]:
# 对训练集进行处理的代码
# 遍历训练集的原始visit txt文件，分9个类别记录每个用户的总时长
# 输出9个txt，每个文件对应一个类别，每个文件中两列记录用户id和时长
# 分两步进行：第一步：遍历txt，第二步：对于每个类别，依据时长对用户排序，删除时长低的80% 或 90% 的用户，再合并同一类别中的相同用户，时长相加

###################################################第一步
id_cnt_files=[]
for i in range(1,10):
    id_cnt_file=open("id_cnt/id_cnt_"+str(i)+".txt","a+")
    id_cnt_files.append(id_cnt_file)
files=getfiles("train")
files.extend(getfiles("val"))
all_cnt=len(files)
print(all_cnt)
cnt=0
last_time=time()
for file in files:
    label=int(file.split(".txt")[0].split("/")[-1].split("_")[1])
    table = pd.read_table(file, header=None)
    tv=table.values
    user_num=tv.shape[0]
    temp=""
    for i in range(user_num):
        temp+=str(tv[i,0])+","+str((tv[i,1].count("&")+tv[i,1].count("|"))*1.0/tv[i,1].count("&")) +"\n"
    id_cnt_files[label-1].write(temp)
    if cnt%1000==0:
        now_time=time()
        print("%.4f%%"%(cnt/all_cnt*100),now_time-last_time)
        last_time=time()
    cnt+=1
close_txt(id_cnt_files)

400000
0.0000% 0.1405012607574463
0.2500% 71.90762543678284
0.5000% 65.54552149772644
0.7500% 64.48962879180908
1.0000% 51.08886766433716
1.2500% 62.38446259498596
1.5000% 64.08905029296875
1.7500% 66.90772104263306
2.0000% 65.17323899269104
2.2500% 58.704699754714966
2.5000% 66.33811902999878
2.7500% 68.01233410835266
3.0000% 68.06507873535156
3.2500% 64.55007433891296
3.5000% 69.89925909042358
3.7500% 66.86839747428894
4.0000% 65.86624193191528
4.2500% 67.81334233283997
4.5000% 66.91785478591919
4.7500% 63.74564027786255
5.0000% 67.198965549469
5.2500% 66.14320850372314
5.5000% 67.45385098457336
5.7500% 64.31830406188965
6.0000% 61.92996525764465
6.2500% 60.466185569763184
6.5000% 61.62728500366211
6.7500% 68.40701627731323
7.0000% 73.9743812084198
7.2500% 67.32410168647766
7.5000% 69.60120224952698
7.7500% 66.4634599685669
8.0000% 66.73422646522522
8.2500% 65.0310640335083
8.5000% 65.99755215644836
8.7500% 69.52451777458191
9.0000% 68.21053528785706
9.2500% 67.62555861473083
9.5000%

76.2500% 71.11020374298096
76.5000% 67.22749948501587
76.7500% 66.33210062980652
77.0000% 68.18577980995178
77.2500% 70.0208306312561
77.5000% 70.31470918655396
77.7500% 68.34680438041687
78.0000% 70.32745599746704
78.2500% 67.39676809310913
78.5000% 64.92605876922607
78.7500% 67.22063493728638
79.0000% 62.71514129638672
79.2500% 67.00409579277039
79.5000% 66.56651258468628
79.7500% 71.716956615448
80.0000% 71.34194731712341
80.2500% 68.10986614227295
80.5000% 69.48819041252136
80.7500% 68.23866844177246
81.0000% 68.06290221214294
81.2500% 68.36642575263977
81.5000% 69.06518483161926
81.7500% 67.89934372901917
82.0000% 69.68445062637329
82.2500% 63.75875926017761
82.5000% 68.10044598579407
82.7500% 73.05597972869873
83.0000% 72.11360549926758
83.2500% 70.29507040977478
83.5000% 67.25213384628296
83.7500% 74.69013500213623
84.0000% 69.0601954460144
84.2500% 66.56307029724121
84.5000% 67.33891153335571
84.7500% 70.54882740974426
85.0000% 66.34927129745483
85.2500% 73.9984359741211
85.500

In [5]:
###################################################第二步
id_cnt_files=[]
for i in range(9,10):
    id_cnt_file=open("id_cnt_del_same/id_cnt_"+str(i)+".txt","a+")
    id_cnt_files.append(id_cnt_file)
    
last_time=time()
for i in range(9,10):
    table = pd.read_table("id_cnt/id_cnt_"+str(i)+".txt",sep=",", header=None)    
    temp=table.values[:,:]
    user_num=temp.shape[0]
    sorted_data=temp[np.lexsort(temp.T)]
#     rate=0.9
    if i in [1,2,7,9]:
        rate=0.9
    else:
        rate=0.8 
    first_user=int(user_num*rate)
    user2cnt={}
    last_user=""
    for j in range(first_user,user_num):
        if (j-first_user)%10000==0:
            now_time=time()
            print(i,(j-first_user)/(user_num-first_user),now_time-last_time)
            last_time=time()
        now_user=sorted_data[j,0]
        if last_user==now_user:
            user2cnt[now_user]+=sorted_data[j,1]
        else:
            user2cnt[now_user]=sorted_data[j,1]
            last_user=now_user
    users=user2cnt.keys()
    for user in users:
        id_cnt_files[i-1].write(str(user)+","+str(user2cnt[user])+"\n")
close_txt(id_cnt_files)

9 0.0 1265.6278562545776
9 0.0009838813667332198 0.016950607299804688
9 0.0019677627334664396 0.015585899353027344
9 0.002951644100199659 0.015079021453857422
9 0.003935525466932879 0.014609575271606445
9 0.004919406833666098 0.015535593032836914
9 0.005903288200399318 0.016127824783325195
9 0.006887169567132537 0.014639854431152344
9 0.007871050933865758 0.015040159225463867
9 0.008854932300598977 0.015234708786010742
9 0.009838813667332197 0.015103816986083984
9 0.010822695034065416 0.016903162002563477
9 0.011806576400798637 0.014489412307739258
9 0.012790457767531856 0.015126943588256836
9 0.013774339134265075 0.015440940856933594
9 0.014758220500998296 0.014595508575439453
9 0.015742101867731517 0.014360189437866211
9 0.016725983234464736 0.01442575454711914
9 0.017709864601197955 0.01439666748046875
9 0.018693745967931174 0.016504526138305664
9 0.019677627334664393 0.01531982421875
9 0.020661508701397612 0.015275001525878906
9 0.02164539006813083 0.01694178581237793
9 0.022629271

9 0.20366344291377647 0.01856398582458496
9 0.2046473242805097 0.01405024528503418
9 0.2056312056472429 0.02109503746032715
9 0.20661508701397613 0.021504878997802734
9 0.20759896838070935 0.01960134506225586
9 0.20858284974744257 0.015267372131347656
9 0.20956673111417579 0.018677234649658203
9 0.210550612480909 0.028605222702026367
9 0.21153449384764222 0.022074222564697266
9 0.21251837521437544 0.02760934829711914
9 0.21350225658110866 0.015618324279785156
9 0.21448613794784188 0.01455998420715332
9 0.2154700193145751 0.014395952224731445
9 0.21645390068130832 0.02108907699584961
9 0.21743778204804154 0.014385223388671875
9 0.21842166341477476 0.01370382308959961
9 0.21940554478150798 0.013703107833862305
9 0.2203894261482412 0.014337539672851562
9 0.22137330751497442 0.01399683952331543
9 0.22235718888170763 0.014215230941772461
9 0.22334107024844088 0.014469385147094727
9 0.2243249516151741 0.014130830764770508
9 0.22530883298190732 0.014533281326293945
9 0.22629271434864054 0.016

9 0.4004397162604204 0.01490020751953125
9 0.4014235976271536 0.018269777297973633
9 0.40240747899388685 0.0189511775970459
9 0.40339136036062007 0.014935731887817383
9 0.4043752417273533 0.01894092559814453
9 0.4053591230940865 0.03313326835632324
9 0.4063430044608197 0.01798391342163086
9 0.40732688582755294 0.027631521224975586
9 0.40831076719428616 0.015064001083374023
9 0.4092946485610194 0.014020204544067383
9 0.4102785299277526 0.01372528076171875
9 0.4112624112944858 0.021123886108398438
9 0.41224629266121904 0.013477087020874023
9 0.41323017402795226 0.013289928436279297
9 0.4142140553946855 0.013447761535644531
9 0.4151979367614187 0.013086080551147461
9 0.4161818181281519 0.013113975524902344
9 0.41716569949488513 0.012609720230102539
9 0.41814958086161835 0.012523174285888672
9 0.41913346222835157 0.013170480728149414
9 0.4201173435950848 0.016107559204101562
9 0.421101224961818 0.018353700637817383
9 0.42208510632855123 0.01974630355834961
9 0.42306898769528445 0.015317678

9 0.6100064473745962 0.013414382934570312
9 0.6109903287413294 0.013618707656860352
9 0.6119742101080626 0.013698816299438477
9 0.6129580914747959 0.013579368591308594
9 0.6139419728415291 0.013281583786010742
9 0.6149258542082623 0.013401985168457031
9 0.6159097355749955 0.015786170959472656
9 0.6168936169417287 0.013568878173828125
9 0.617877498308462 0.013360738754272461
9 0.6188613796751952 0.013264894485473633
9 0.6198452610419284 0.01717376708984375
9 0.6208291424086616 0.014193296432495117
9 0.6218130237753948 0.014139413833618164
9 0.622796905142128 0.013714075088500977
9 0.6237807865088613 0.014127731323242188
9 0.6247646678755945 0.014754295349121094
9 0.6257485492423277 0.014681339263916016
9 0.6267324306090609 0.014199972152709961
9 0.6277163119757941 0.013765096664428711
9 0.6287001933425274 0.013968944549560547
9 0.6296840747092606 0.014557600021362305
9 0.6306679560759938 0.014559268951416016
9 0.631651837442727 0.01592421531677246
9 0.6326357188094602 0.0147290229797363

9 0.8185892971220388 0.015150308609008789
9 0.819573178488772 0.015077829360961914
9 0.8205570598555052 0.01453709602355957
9 0.8215409412222384 0.014429807662963867
9 0.8225248225889716 0.014816761016845703
9 0.8235087039557049 0.01644420623779297
9 0.8244925853224381 0.014816761016845703
9 0.8254764666891713 0.01475834846496582
9 0.8264603480559045 0.014818668365478516
9 0.8274442294226377 0.014226913452148438
9 0.828428110789371 0.01418447494506836
9 0.8294119921561042 0.014404773712158203
9 0.8303958735228374 0.01437997817993164
9 0.8313797548895706 0.014119386672973633
9 0.8323636362563038 0.013819456100463867
9 0.833347517623037 0.013916254043579102
9 0.8343313989897703 0.014236927032470703
9 0.8353152803565035 0.014129400253295898
9 0.8362991617232367 0.014146566390991211
9 0.8372830430899699 0.014376163482666016
9 0.8382669244567031 0.014142036437988281
9 0.8392508058234364 0.013906240463256836
9 0.8402346871901696 0.01537322998046875
9 0.8412185685569028 0.017829179763793945
9

IndexError: list index out of range

In [ ]:
# # 验证集代码
# ##################################################从id_cnt_del_same文件夹读取训练集的用户时长，存入9个字典中 user2hours
# user2hours=[{} for i in range(9)]
# for i in range(1,10):
#     label=i
#     table = pd.read_table("id_cnt_del_same/id_cnt_"+str(i)+".txt",sep=",", header=None)    
#     temp=table.values[:,:]
#     user_num=temp.shape[0]
#     sorted_data=temp[np.lexsort(temp.T)]
#     for j in range(user_num):
#         if (j)%50000==0:
#             print(i,(j)/(user_num))
#         now_user=sorted_data[j,0]
#         user2hours[i-1][now_user]=int(sorted_data[j,1])
# ################################################### 根据user2hours记录老用户的时长，
# # 因为字典操作需要保证key值存在，所以需要二分搜索key (用户id)是否是在字典的keys中出现过

# # id2label=np.zeros([50000])
# # id2pro=np.zeros([50000,9])
# # id2num=np.zeros([50000,9])

# id2label={}
# id2pro={}
# id2num={}
# all_keys=[]  #记录所有训练集用户id，用于后续二分查询
# for i in range(9):
#     temp=list(user2hours[i].keys())
#     temp.sort()
#     all_keys.append(temp[:])
# files=getfiles("val")
# all_cnt=len(files)
# print(all_cnt)
# cnt=0
# acc_cnt=0
# last_time=time()
# for file in files:
#     s=file.split(".txt")[0].split("/")[-1].split("_")
#     reg_id=s[0]
#     true_label=int(s[1])
#     table = pd.read_table(file, header=None)
#     table2cnt=table.values[:,:]
#     user_num=table2cnt.shape[0]
#     for i in range(user_num):
#         now_user_hours=(table2cnt[i,1].count("&")+table2cnt[i,1].count("|"))*1.0/table2cnt[i,1].count("&")
#         table2cnt[i,1]=now_user_hours
#     temp=table2cnt[:,:]
#     sorted_data=temp[np.lexsort(temp.T)]
#     rate=0.5   #剔除单个测试样本中时长较短的用户，剔除比例为rate
#     first_user=int(user_num*rate)
#     reg_per_label=np.zeros([9])
#     for i in range(first_user,user_num):
#         now_user=str(sorted_data[i,0])
#         hours_per_label=np.zeros([9])
#         for j in range(9):
#             if find_key(all_keys[j],now_user):
#                 hours_per_label[j]=user2hours[j][now_user]
#         reg_per_label+=hours_per_label
#     id2num[reg_id]=reg_per_label
#     pro=cnt2pro(reg_per_label)
#     pre_label=np.argmax(pro)+1
#     if np.sum(reg_per_label)==0:  #该测试样本内老用户为0，用类别 -1 标记缺失
#         pre_label=-1
#     id2label[reg_id]=pre_label
#     id2pro[reg_id]=pro[:]
#     if true_label==pre_label:
#         acc_cnt+=1
#     cnt+=1
#     if cnt%1000==0:
#         now_time=time()
#         print("%.4f%%"%(cnt/all_cnt*100),"val acc ",acc_cnt/cnt,now_time-last_time)
#         last_time=time()

# # np.save("id2num_val.npy",id2num)
# # np.save("id2label_val.npy",id2label)
# # np.save("id2pro_val.npy",id2pro)
# # files=getfiles('val')
# files.sort()
# f_list=[]
# for file in files:
#     s=file.split('.txt')[0].split('/')[-1].split('_')[0]
#     f_list.append(s)
# id2num_val=open('id2num_val.txt','w')
# for key in f_list:
#     line=key+','+str(id2num[key][0])+','+str(id2num[key][1])+','+str(id2num[key][2])+','+str(id2num[key][3])+','+str(id2num[key][4])+','+str(id2num[key][5])+','+str(id2num[key][6])+','+str(id2num[key][7])+','+str(id2num[key][8])+'\n'
#     id2num_val.write(line)
# id2num_val.close()
# print(1)
# id2label_val=open('id2label_val.txt','w')
# for key in f_list:
#     line=key+','+str(id2label[key])+'\n'
#     id2label_val.write(line)
# id2label_val.close()
# print(2)
# id2pro_val=open('id2pro_val.txt','w')
# for key in f_list:
#     line=key+','+str(id2pro[key][0])+','+str(id2pro[key][1])+','+str(id2pro[key][2])+','+str(id2pro[key][3])+','+str(id2pro[key][4])+','+str(id2pro[key][5])+','+str(id2pro[key][6])+','+str(id2pro[key][7])+','+str(id2pro[key][8])+'\n'
#     id2pro_val.write(line)
# id2pro_val.close()
# print(3)

In [ ]:
# 测试代码
##################################################从id_cnt_del_same文件夹读取训练集的用户时长，存入9个字典中 user2hours
user2hours=[{} for i in range(9)]
for i in range(1,10):
    label=i
    table = pd.read_table("id_cnt_del_same/id_cnt_"+str(i)+".txt",sep=",", header=None)    
    temp=table.values[:,:]
    user_num=temp.shape[0]
    sorted_data=temp[np.lexsort(temp.T)]
    for j in range(user_num):
        if (j)%50000==0:
            print(i,(j)/(user_num))
        now_user=sorted_data[j,0]
        user2hours[i-1][now_user]=int(sorted_data[j,1])
################################################### 根据user2hours记录老用户的时长，
# 因为字典操作需要保证key值存在，所以需要二分搜索key (用户id)是否是在字典的keys中出现过

id2label=np.zeros([100000])
id2pro=np.zeros([100000,9])
id2num=np.zeros([100000,9])
all_keys=[]  #记录所有训练集用户id，用于后续二分查询
for i in range(9):
    temp=list(user2hours[i].keys())
    temp.sort()
    all_keys.append(temp[:])
files=getfiles("test")
all_cnt=len(files)
print(all_cnt)
cnt=0
last_time=time()
for file in files:
    reg_id=int(file.split(".txt")[0].split("/")[-1])
    table = pd.read_table(file, header=None)
    table2cnt=table.values[:,:]
    user_num=table2cnt.shape[0]
    for i in range(user_num):
        now_user_hours=(table2cnt[i,1].count("&")+table2cnt[i,1].count("|"))*1.0/table2cnt[i,1].count("&")
        table2cnt[i,1]=now_user_hours
    temp=table2cnt[:,:]
    sorted_data=temp[np.lexsort(temp.T)]
    rate=0.5   #剔除单个测试样本中时长较短的用户，剔除比例为rate
    first_user=int(user_num*rate)
    reg_per_label=np.zeros([9])
    for i in range(first_user,user_num):
        now_user=str(sorted_data[i,0])
        hours_per_label=np.zeros([9])
        for j in range(9):
            if find_key(all_keys[j],now_user):
                hours_per_label[j]=user2hours[j][now_user]
        reg_per_label+=hours_per_label
    id2num[reg_id]=reg_per_label
    pro=cnt2pro(reg_per_label)
    pre_label=np.argmax(pro)+1
    if np.sum(reg_per_label)==0:  #该测试样本内老用户为0，用类别 -1 标记缺失
        pre_label=-1
    id2label[reg_id]=pre_label
    id2pro[reg_id]=pro[:]
    if cnt%1000==0:
        now_time=time()
        print("%.4f%%"%(cnt/all_cnt*100),now_time-last_time)
        last_time=time()
    cnt+=1

np.save("id2num_test.npy",id2num)
np.save("id2label_test.npy",id2label)
np.save("id2pro_test.npy",id2pro)